In [1]:
import numpy as np
import pandas as pd
import tiktoken

In [2]:
tokenizer = tiktoken.get_encoding("cl100k_base")

In [24]:
text = "My name is Anoop!"
tokens = tokenizer.encode(text)
print("Tokens:", tokens)
decoded_text = tokenizer.decode(tokens)
print("Decoded Text:", decoded_text)

Tokens: [5159, 836, 374, 1556, 48406, 0]
Decoded Text: My name is Anoop!


In [3]:
qa_data = pd.read_csv(r'D:\ML_Projects\LLM-From-Scratch-For-ChatBots-GPT2\Data\ChatGPT_chatlogs\GPT_chatlogs_Q_Tag_12.6K.csv')

In [30]:
chat_list = []
j = 0
for idx, row in qa_data.iterrows():
    chat_list.append(len(tokenizer.encode(row['Question'])))
    chat_list.append(len(tokenizer.encode(row['Answer'])))
print(min(chat_list))
print(max(chat_list))

4
782


In [38]:
qa_data['Q_Encoding_Len'] = qa_data['Question'].apply(lambda x: tokenizer.encode(x))
qa_data['A_Encoding_Len'] = qa_data['Answer'].apply(lambda x: tokenizer.encode(x))

qa_data.to_csv(r'D:\ML_Projects\LLM-From-Scratch-For-ChatBots-GPT2\Data\ChatGPT_chatlogs\GPT_chatlogs_Q_Tag_12.6K_Tokens.csv', index=False)
qa_data.head()

,Question,Answer,Q_Encoding_Len,A_Encoding_Len
0,Would you like to play a game,Sure! I'd be happy to play a game with you. Do...,"[29089, 499, 1093, 311, 1514, 264, 1847]","[40914, 0, 358, 4265, 387, 6380, 311, 1514, 26..."
1,What are the other questions that I can ask them,Here are some other questions that you can ask...,"[3923, 527, 279, 1023, 4860, 430, 358, 649, 26...","[8586, 527, 1063, 1023, 4860, 430, 499, 649, 2..."
2,what is stellar abberation,Stellar aberration is the apparent change in t...,"[12840, 374, 48317, 671, 655, 367]","[626, 27978, 82102, 367, 374, 279, 10186, 2349..."
3,What are some readings for week 6,Certainly! Here are some alternative course ti...,"[3923, 527, 1063, 40174, 369, 2046, 220, 21]","[96556, 0, 5810, 527, 1063, 10778, 3388, 15671..."
4,"can you provide a reading list for ""The Coldes...",Certainly! Here are some suggested instruction...,"[4919, 499, 3493, 264, 5403, 1160, 369, 330, 7...","[96556, 0, 5810, 527, 1063, 12090, 11470, 369,..."


In [42]:
def get_total_encoding_length(row):
    question_length = len(tokenizer.encode(row['Question']))
    answer_length = len(tokenizer.encode(row['Answer']))
    return question_length + answer_length

qa_data['QA_Encoding_Len'] = qa_data.apply(get_total_encoding_length, axis=1)

qa_data = qa_data[qa_data['QA_Encoding_Len']<=100]
qa_data.drop('QA_Encoding_Len', inplace=True, axis=1)

qa_data.to_csv(r'D:\ML_Projects\LLM-From-Scratch-For-ChatBots-GPT2\Data\ChatGPT_chatlogs\GPT_chatlogs_Q_Tag_11.9K.csv', index=False)
qa_data.head()

,Question,Answer
0,Would you like to play a game,Sure! I'd be happy to play a game with you. Do...
1,What are the other questions that I can ask them,Here are some other questions that you can ask...
2,what is stellar abberation,Stellar aberration is the apparent change in t...
3,What are some readings for week 6,Certainly! Here are some alternative course ti...
4,"can you provide a reading list for ""The Coldes...",Certainly! Here are some suggested instruction...


In [ ]:
qa_data = pd.read_csv(r'D:\ML_Projects\LLM-From-Scratch-For-ChatBots-GPT2\Data\ChatGPT_chatlogs\GPT_chatlogs_Q_Tag_11.9K.csv')
tokenizer = tiktoken.get_encoding("cl100k_base")

for row in qa_data.itertuples():
    chat = row.Question + "\n" + row.Answer
    tokens = tokenizer.encode(chat)
    padded_tokens = tokens + [0] * (100 - len(tokens)) if len(tokens) < 100 else tokens[:100]

In [ ]:
import pandas as pd
import torch
import tiktoken

device = 'cuda' if torch.cuda.is_available() else 'cpu'

batch_size = 4
block_size = 1024

tokenizer = tiktoken.get_encoding("cl100k_base")
qa_data = pd.read_csv(r'D:\ML_Projects\LLM-From-Scratch-For-ChatBots-GPT2\Data\ChatGPT_chatlogs\GPT_chatlogs_Q_Tag_11.9K.csv')

def process_data(qa_data, tokenizer):
    tokens = []
    for row in qa_data.itertuples():
        chat = row.Question + "\n" + row.Answer
        tokens.append(tokenizer.encode(chat))
    return tokens

def get_batch_tiktoken(qa_tokenized):
    X = []
    Y = []
    for i in range(batch_size):
        rand_row = torch.randint(0, len(qa_tokenized), (1,))
        token_len = block_size + 1
        tokens = qa_tokenized[rand_row]
        padded_tokens = tokens + [0] * (token_len - len(tokens)) if len(tokens) < token_len else tokens[:token_len]
        X.append(padded_tokens[:-1])
        Y.append(padded_tokens[1:])
    x_batch = torch.tensor(X, dtype=torch.long)
    y_batch = torch.tensor(Y, dtype=torch.long)
    return x_batch.to(device), y_batch.to(device)

qa_tokenized = process_data(qa_data, tokenizer)
batch_x, batch_y = get_batch_tiktoken(qa_tokenized)

print('batch_x len: ', len(batch_x))

for i in range(batch_size):
    print(f'batch_x[{i}] len: ', len(batch_x[i]))

In [4]:
qa_data = pd.read_csv(r'D:\ML_Projects\LLM-From-Scratch-For-ChatBots-GPT2\Data\ChatGPT_chatlogs\GPT_chatlogs_Q_Tag_11.9K.csv')

tokenizer = tiktoken.get_encoding("cl100k_base")
unique_tokens = set()

for row in qa_data.itertuples():
    chat = row.Question + "\n" + row.Answer
    tokens = tokenizer.encode(chat)
    unique_tokens.update(tokens)

In [5]:
print(len(unique_tokens))
print(max(unique_tokens))

26467
100235


In [ ]:
#LLM_run.py - for Char level

import torch
import sys

sys.path.append(r'D:\ML_Projects\LLM-From-Scratch-For-ChatBots-GPT2\Scripts')
from LLM_Finetune_Char import Head, MultiHeadAttention, FeedFoward, Block, GPTLanguageModel, encode, decode, string_to_int, int_to_string

device = 'cuda' if torch.cuda.is_available() else 'cpu'

model = torch.load(r'D:\ML_Projects\LLM-From-Scratch-For-ChatBots-GPT2\Models\model-06.pth')
model.eval()
model.to(device)

while True:
    prompt = input("Prompt (enter 'x' to exit):\n")
    if prompt.lower() == "x":
        break
    context = torch.tensor(encode(prompt), dtype=torch.long, device=device)
    generated_chars = decode(model.generate(context.unsqueeze(0), max_new_tokens=150)[0].tolist())
    print(f'Generated text:\n{generated_chars}')

In [1]:
import pandas as pd
import tiktoken

In [5]:
qa_data = pd.read_excel(r'D:\ML_Projects\LLM-From-Scratch-For-ChatBots-GPT2\Data\SQuAD\SQuAD_Preprocessed.xlsx')

tokenizer = tiktoken.get_encoding("cl100k_base")
qa_tokens = []
for row in qa_data.itertuples():
    chat = "Context: " + row.Context + " [SEP] Question: " + row.Question + " [SEP] Answer: " + str(row.Answer)
    tokens = tokenizer.encode(chat)
    qa_tokens.append(tokens)